In [1]:
import tensorflow as tf

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.25)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# tf.config.gpu.set_per_process_memory_fraction(0.25)
# tf.config.gpu.set_per_process_memory_growth(True)

# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth=True
# sess = tf.compat.v1.Session(config=config)

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.35
session = tf.compat.v1.Session(config=config)

# tf.config.gpu.set_per_process_memory_growth(True)
# tf.config.experimental.set_virtual_device_configuration(0.25)

2023-02-17 15:04:42.681315: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-02-17 15:04:43.651055: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 15:04:43.651381: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-17 15:04:43.652248: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-02-17 15:04:43.709459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node ze

In [2]:
"""
 ARL_EEGModels - A collection of Convolutional Neural Network models for EEG
 Signal Processing and Classification, using Keras and Tensorflow

 Requirements:
    (1) tensorflow == 2.X (as of this writing, 2.0 - 2.3 have been verified
        as working)
 
 To run the EEG/MEG ERP classification sample script, you will also need

    (4) mne >= 0.17.1
    (5) PyRiemann >= 0.2.5
    (6) scikit-learn >= 0.20.1
    (7) matplotlib >= 2.2.3
    
 To use:
    
    (1) Place this file in the PYTHONPATH variable in your IDE (i.e.: Spyder)
    (2) Import the model as
        
        from EEGModels import EEGNet    
        
        model = EEGNet(nb_classes = ..., Chans = ..., Samples = ...)
        
    (3) Then compile and fit the model
    
        model.compile(loss = ..., optimizer = ..., metrics = ...)
        fitted    = model.fit(...)
        predicted = model.predict(...)

 Portions of this project are works of the United States Government and are not
 subject to domestic copyright protection under 17 USC Sec. 105.  Those 
 portions are released world-wide under the terms of the Creative Commons Zero 
 1.0 (CC0) license.  
 
 Other portions of this project are subject to domestic copyright protection 
 under 17 USC Sec. 105.  Those portions are licensed under the Apache 2.0 
 license.  The complete text of the license governing this material is in 
 the file labeled LICENSE.TXT that is a part of this project's official 
 distribution. 
"""

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K


def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta

    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:
        
        1. Depthwise Convolutions to learn spatial filters within a 
        temporal convolution. The use of the depth_multiplier option maps 
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn 
        spatial filters within each filter in a filter-bank. This also limits 
        the number of free parameters to fit when compared to a fully-connected
        convolution. 
        
        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions. 
        
    
    While the original paper used Dropout, we found that SpatialDropout2D 
    sometimes produced slightly better results for classification of ERP 
    signals. However, SpatialDropout2D significantly reduced performance 
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.
        
    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the 
    kernel lengths for double the sampling rate, etc). Note that we haven't 
    tested the model performance with this rule so this may not work well. 
    
    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.

    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D. 

    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.

    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)




def EEGNet_SSVEP(nb_classes = 12, Chans = 8, Samples = 256, 
             dropoutRate = 0.5, kernLength = 256, F1 = 96, 
             D = 1, F2 = 96, dropoutType = 'Dropout'):
    """ SSVEP Variant of EEGNet, as used in [1]. 

    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. 
      D               : number of spatial filters to learn within each temporal
                        convolution.
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
      
      
    [1]. Waytowich, N. et. al. (2018). Compact Convolutional Neural Networks
    for Classification of Asynchronous Steady-State Visual Evoked Potentials.
    Journal of Neural Engineering vol. 15(6). 
    http://iopscience.iop.org/article/10.1088/1741-2552/aae5d8

    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)



def EEGNet_old(nb_classes, Chans = 64, Samples = 128, regRate = 0.0001,
           dropoutRate = 0.25, kernels = [(2, 32), (8, 4)], strides = (2, 4)):
    """ Keras Implementation of EEGNet_v1 (https://arxiv.org/abs/1611.08024v2)

    This model is the original EEGNet model proposed on arxiv
            https://arxiv.org/abs/1611.08024v2
    
    with a few modifications: we use striding instead of max-pooling as this 
    helped slightly in classification performance while also providing a 
    computational speed-up. 
    
    Note that we no longer recommend the use of this architecture, as the new
    version of EEGNet performs much better overall and has nicer properties.
    
    Inputs:
        
        nb_classes     : total number of final categories
        Chans, Samples : number of EEG channels and samples, respectively
        regRate        : regularization rate for L1 and L2 regularizations
        dropoutRate    : dropout fraction
        kernels        : the 2nd and 3rd layer kernel dimensions (default is 
                         the [2, 32] x [8, 4] configuration)
        strides        : the stride size (note that this replaces the max-pool
                         used in the original paper)
    
    """

    # start the model
    input_main   = Input(shape = (Chans, Samples, 1))
    layer1       = Conv2D(16, (Chans, 1), input_shape=(Chans, Samples, 1),
                                 kernel_regularizer = l1_l2(l1=regRate, l2=regRate))(input_main)
    layer1       = BatchNormalization()(layer1)
    layer1       = Activation('elu')(layer1)
    layer1       = Dropout(dropoutRate)(layer1)
    
    permute_dims = 2, 1, 3
    permute1     = Permute(permute_dims)(layer1)
    
    layer2       = Conv2D(4, kernels[0], padding = 'same', 
                            kernel_regularizer=l1_l2(l1=0.0, l2=regRate),
                            strides = strides)(permute1)
    layer2       = BatchNormalization()(layer2)
    layer2       = Activation('elu')(layer2)
    layer2       = Dropout(dropoutRate)(layer2)
    
    layer3       = Conv2D(4, kernels[1], padding = 'same',
                            kernel_regularizer=l1_l2(l1=0.0, l2=regRate),
                            strides = strides)(layer2)
    layer3       = BatchNormalization()(layer3)
    layer3       = Activation('elu')(layer3)
    layer3       = Dropout(dropoutRate)(layer3)
    
    flatten      = Flatten(name = 'flatten')(layer3)
    
    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)



def DeepConvNet(nb_classes, Chans = 64, Samples = 256,
                dropoutRate = 0.5):
    """ Keras implementation of the Deep Convolutional Network as described in
    Schirrmeister et. al. (2017), Human Brain Mapping.
    
    This implementation assumes the input is a 2-second EEG signal sampled at 
    128Hz, as opposed to signals sampled at 250Hz as described in the original
    paper. We also perform temporal convolutions of length (1, 5) as opposed
    to (1, 10) due to this sampling rate difference. 
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                      ours        original paper
    pool_size        1, 2        1, 3
    strides          1, 2        1, 3
    conv filters     1, 5        1, 10
    
    Note that this implementation has not been verified by the original 
    authors. 
    
    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(25, (1, 5), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(25, (Chans, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1       = Activation('elu')(block1)
    block1       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block1)
    block1       = Dropout(dropoutRate)(block1)
  
    block2       = Conv2D(50, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block2       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block2)
    block2       = Activation('elu')(block2)
    block2       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block2)
    block2       = Dropout(dropoutRate)(block2)
    
    block3       = Conv2D(100, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block2)
    block3       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block3)
    block3       = Activation('elu')(block3)
    block3       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block3)
    block3       = Dropout(dropoutRate)(block3)
    
    block4       = Conv2D(200, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block3)
    block4       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block4)
    block4       = Activation('elu')(block4)
    block4       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block4)
    block4       = Dropout(dropoutRate)(block4)
    
    flatten      = Flatten()(block4)
    
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)


# need these for ShallowConvNet
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   


def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    """ Keras implementation of the Shallow Convolutional Network as described
    in Schirrmeister et. al. (2017), Human Brain Mapping.
    
    Assumes the input is a 2-second EEG signal sampled at 128Hz. Note that in 
    the original paper, they do temporal convolutions of length 25 for EEG
    data sampled at 250Hz. We instead use length 13 since the sampling rate is 
    roughly half of the 250Hz which the paper used. The pool_size and stride
    in later layers is also approximately half of what is used in the paper.
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                     ours        original paper
    pool_size        1, 35       1, 75
    strides          1, 7        1, 15
    conv filters     1, 13       1, 25    
    
    Note that this implementation has not been verified by the original 
    authors. We do note that this implementation reproduces the results in the
    original paper with minor deviations. 
    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(40, (1, 13), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                          kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7))(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)



In [3]:
import sys

lag = 10 #int(str(sys.argv[1]))

In [4]:
from scipy.io import loadmat

mat_contents = loadmat('song21_Imputed.mat')
mat_contents.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data21', 'fs', 'subs21'])

In [5]:
event_id = dict(s1=0, s2=1, s3=2, s4=3)

In [6]:
subj = mat_contents['subs21']

In [7]:
subj.shape

(1, 20)

In [8]:
subj

array([[array(['S02'], dtype='<U3'), array(['S03'], dtype='<U3'),
        array(['S04'], dtype='<U3'), array(['S05'], dtype='<U3'),
        array(['S06'], dtype='<U3'), array(['S07'], dtype='<U3'),
        array(['S08'], dtype='<U3'), array(['S09'], dtype='<U3'),
        array(['S10'], dtype='<U3'), array(['S11'], dtype='<U3'),
        array(['S12'], dtype='<U3'), array(['S13'], dtype='<U3'),
        array(['S14'], dtype='<U3'), array(['S15'], dtype='<U3'),
        array(['S16'], dtype='<U3'), array(['S17'], dtype='<U3'),
        array(['S19'], dtype='<U3'), array(['S20'], dtype='<U3'),
        array(['S21'], dtype='<U3'), array(['S23'], dtype='<U3')]],
      dtype=object)

In [12]:
sub_ids = []
for id in subj[0]:
    sub_ids.append(int(id[0][1:]))

In [10]:
sub_ids

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23]

In [10]:
data = mat_contents['data21']

In [11]:
data.shape

(125, 34795, 20)

In [12]:
sub_data = []
x = []
y = []
z = 125*lag
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,0])

In [13]:
mat_contents = loadmat('song22_Imputed.mat')
mat_contents.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data22', 'fs', 'subs22'])

In [14]:
subj = mat_contents['subs22']
sub_ids = []
for id in subj[0]:
    sub_ids.append(int(id[0][1:]))
print(sub_ids)
data = mat_contents['data22']
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,1])

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23]


In [15]:
mat_contents = loadmat('song23_Imputed.mat')
mat_contents.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data23', 'fs', 'subs23'])

In [16]:
subj = mat_contents['subs23']
sub_ids = []
for id in subj[0]:
    sub_ids.append(int(id[0][1:]))
print(sub_ids)
data = mat_contents['data23']
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,2])

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23]


In [17]:
mat_contents = loadmat('song24_Imputed.mat')
mat_contents.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data24', 'fs', 'subs24'])

In [18]:
subj = mat_contents['subs24']
sub_ids = []
for id in subj[0]:
    sub_ids.append(int(id[0][1:]))
print(sub_ids)
data = mat_contents['data24']
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,3])

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23]


In [19]:
mat_contents = loadmat('song25_Imputed.mat')
data = mat_contents['data25']
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,4])
mat_contents = loadmat('song26_Imputed.mat')
data = mat_contents['data26']
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,5])
mat_contents = loadmat('song27_Imputed.mat')
data = mat_contents['data27']
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,6])
mat_contents = loadmat('song28_Imputed.mat')
data = mat_contents['data28']
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,7])
mat_contents = loadmat('song29_Imputed.mat')
data = mat_contents['data29']
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,8])
mat_contents = loadmat('song30_Imputed.mat')
data = mat_contents['data30']
for idx in range(20):
    for i in range(0,len(data[0])-z,z):
        x.append(data[:,i:i+(z),idx])
        y.append([idx,9])

In [20]:
data.shape

(125, 37221, 20)

In [21]:
len(x)

5620

In [22]:
len(y)

5620

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
import numpy as np
from matplotlib import pyplot as plt
from pyriemann.utils.viz import plot_confusion_matrix
import pandas as pd

y = np.array(y)
x =np.array(x)
chans = x.shape[1]
samples = x.shape[2]
kernels = 1
print(x.shape)
print(y.shape)
print(set(y[:,0]))
print(set(y[:,1]))

clfs_names = ['EEGNet','EEGNet_SSVEP','DeepConvNet']

df_values = pd.DataFrame()

class_weights = {0:1, 1:1, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1, 8:1, 9:1}
kfold = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
for mdx in clfs_names:
    acc = []
    prec = []
    rec = []
    f1 = []
    for train_index, test_index in kfold.split(x,y[:,1]):
        
        if mdx == 'EEGNet':
            model = EEGNet(nb_classes = 10, Chans = chans, Samples = samples, 
                           dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16, dropoutType = 'Dropout')
        elif mdx == 'EEGNet_SSVEP':
            model = EEGNet_SSVEP(nb_classes = 10, Chans = chans, Samples = samples,
                                 dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16, dropoutType = 'Dropout')
        else:
            model = DeepConvNet(nb_classes = 10, Chans = chans, Samples = samples, dropoutRate = 0.5)
            
            
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train,stratify=y_train[:,1],test_size=0.12)

        # convert labels to one-hot encodings.
        Y_train      = np_utils.to_categorical(y_train[:,1])
        Y_validate   = np_utils.to_categorical(y_validate[:,1])
        Y_test       = np_utils.to_categorical(y_test[:,1])

        # convert data to NHWC (trials, channels, samples, kernels) format. Data 
        # contains 60 channels and 151 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
        X_validate   = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
        X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)

        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
        
        # count number of parameters in the model
        numParams    = model.count_params()
        
        # set a valid path for your system to record model checkpoints
        checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint_120.h5', verbose=1, save_best_only=True)
        fittedModel = model.fit(X_train, Y_train, batch_size = 32, epochs = 2, 
                            verbose = 2, validation_data=(X_validate, Y_validate),
                            callbacks=[checkpointer], class_weight = class_weights)
        model.load_weights('/tmp/checkpoint_120.h5')

        probs       = model.predict(X_test)
        preds       = probs.argmax(axis = -1)  
        ac         = np.mean(preds == Y_test.argmax(axis=-1))
        acc.append(ac)
        prec.append(precision_score(Y_test.argmax(axis=-1), preds, average='macro'))
        rec.append(recall_score(Y_test.argmax(axis=-1), preds, average='macro'))
        f1.append(f1_score(Y_test.argmax(axis=-1), preds, average='macro'))
        print("Classification accuracy: %f " % (ac))
        print("Recall: %f" % recall_score(Y_test.argmax(axis=-1), preds, average='macro'))
        print("Precision: %f" % precision_score(Y_test.argmax(axis=-1), preds, average='macro'))
        print("Accuracy: %f" % accuracy_score(Y_test.argmax(axis=-1), preds, ))
        print("F1 Score: %f" % f1_score(Y_test.argmax(axis=-1), preds, average='macro'))


        names        = ['song1', 'song2', 'song3', 'song4', 'song5', 'song6', 'song7', 'song8', 'song9', 'song10']
        plt.figure(0)
        plot_confusion_matrix(preds, Y_test.argmax(axis = -1), names, title = mdx)
        
    df_values = df_values.append({'Architecture': mdx,
                                                'X': x.shape,
                                                'X_train': X_train.shape,
                                                'X_test': X_test.shape,
                                                'Count_Y_train': len(set(y_train[:,1])),
                                                'Count_Y_test': len(set(y_test[:,1])),
                                                'Accuracies': acc,
                                                'Mean_Accuracy': np.mean(acc),
                                                'Std_Accuracy': np.std(acc),
                                                'Precisions': prec,
                                                'Mean_Precision': np.mean(prec),
                                                'Std_Precision': np.std(prec),
                                                'Recalls': rec,
                                                'Mean_Recall': np.mean(rec),
                                                'Std_Recall': np.std(rec),
                                                'F1 scores': f1, 
                                                'Mean_F1': np.mean(f1),
                                                'Std_F1': np.std(f1),
                                                'Accuracy_Std': str(np.round(np.mean(acc), 2))+' ±'+str(np.round(np.std(acc), 2))}, ignore_index=True)
    df_values.to_csv('clf_results_NMED-T_5fold_'+str(lag)+'chunk.csv', index=False)
        

(5620, 125, 1250)
(5620, 2)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


2022-07-15 14:07:13.140875: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-15 14:07:13.141060: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-15 14:07:13.141519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: GeForce RTX 2060 SUPER computeCapability: 7.5
coreClock: 1.695GHz coreCount: 34 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-07-15 14:07:13.141543: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-15 14:07:13.141564: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-07-15 14:07:13.141573: I tensorflow/stream_executor/platform/

Epoch 1/2


2022-07-15 14:07:16.867384: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-07-15 14:07:17.042593: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


124/124 - 6s - loss: 2.3095 - accuracy: 0.1044 - val_loss: 2.3035 - val_accuracy: 0.1111

Epoch 00001: val_loss improved from inf to 2.30354, saving model to /tmp/checkpoint_120.h5
Epoch 2/2
124/124 - 4s - loss: 2.2777 - accuracy: 0.1453 - val_loss: 2.3017 - val_accuracy: 0.1259

Epoch 00002: val_loss improved from 2.30354 to 2.30170, saving model to /tmp/checkpoint_120.h5
Classification accuracy: 0.118327 
Recall: 0.118662
Precision: 0.118382
Accuracy: 0.118327
F1 Score: 0.116260
Epoch 1/2
124/124 - 4s - loss: 2.3098 - accuracy: 0.1168 - val_loss: 2.3023 - val_accuracy: 0.1111

Epoch 00001: val_loss improved from inf to 2.30231, saving model to /tmp/checkpoint_120.h5
Epoch 2/2
124/124 - 4s - loss: 2.2766 - accuracy: 0.1408 - val_loss: 2.3101 - val_accuracy: 0.1130

Epoch 00002: val_loss did not improve from 2.30231
Classification accuracy: 0.137011 
Recall: 0.138303
Precision: 0.126640
Accuracy: 0.137011
F1 Score: 0.127628
Epoch 1/2
124/124 - 4s - loss: 2.3144 - accuracy: 0.1034 - val

/tmp/ipykernel_22167/3938278297.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_values = df_values.append({'Architecture': mdx,


Epoch 1/2
124/124 - 4s - loss: 2.4125 - accuracy: 0.1082 - val_loss: 2.3051 - val_accuracy: 0.0741

Epoch 00001: val_loss improved from inf to 2.30515, saving model to /tmp/checkpoint_120.h5
Epoch 2/2
124/124 - 4s - loss: 2.3334 - accuracy: 0.1132 - val_loss: 2.3107 - val_accuracy: 0.1019

Epoch 00002: val_loss did not improve from 2.30515
Classification accuracy: 0.096975 
Recall: 0.095953
Precision: 0.089706
Accuracy: 0.096975
F1 Score: 0.086698
Epoch 1/2


In [29]:
# model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 125, 1250, 1)]    0         
_________________________________________________________________
conv2d (Conv2D)              (None, 125, 1250, 8)      256       
_________________________________________________________________
batch_normalization (BatchNo (None, 125, 1250, 8)      32        
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 1, 1250, 16)       2000      
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 1250, 16)       64        
_________________________________________________________________
activation (Activation)      (None, 1, 1250, 16)       0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 312, 16)        0     

In [30]:
# out = model.get_layer(name ="dense" ).output
# inp = model.input

In [31]:
# model.input

<KerasTensor: shape=(None, 125, 1250, 1) dtype=float32 (created by layer 'input_1')>

In [32]:
# # from keras import backend as K

# inp = model.input                                           # input placeholder
# outputs = [layer.output for layer in model.layers]          # all layer outputs
# functors = [K.function([inp], [out]) for out in outputs]    # evaluation functions


In [ ]:
# # Testing
# test = X_train
# layer_outs = [func([test]) for func in functors]
# # print (layer_outs)

In [ ]:
# print(len(layer_outs))

In [ ]:
# layer_outs[15]

In [ ]:
# layer_outs[14]

In [ ]:
# layer_outs[13]

In [ ]:
# from sklearn.manifold import TSNE
# import seaborn as sns
# rt = np.array(layer_outs[13])
# print(rt.shape)
# X_embedded = TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(rt[0])
# print(X_embedded.shape)

In [ ]:
# T1.argmax(axis=-1)

In [ ]:
# T1.argmax(axis=-1).shape

In [ ]:
# import matplotlib.pyplot as plt

# fig = plt.gcf()

# # Change seaborn plot size
# fig.set_size_inches(11, 14)
# sns.scatterplot(x = X_embedded[:,0], y = X_embedded[:,1], hue = Y_train.argmax(axis=-1), palette = sns.hls_palette(10), legend = 'full')

In [ ]:
# import matplotlib.pyplot as plt

# fig = plt.gcf()

# # Change seaborn plot size
# fig.set_size_inches(11, 14)
# sns.scatterplot(x = X_embedded[:,0], y = X_embedded[:,1], hue = y_train[:,0], palette = sns.hls_palette(20), legend = 'full')

In [ ]:
# # from keras import backend as K

# inp = model.input                                           # input placeholder
# outputs = [layer.output for layer in model.layers]          # all layer outputs
# functors = [K.function([inp], [out]) for out in outputs]    # evaluation functions


In [ ]:
# # Testing
# test = X_test
# layer_outs = [func([test]) for func in functors]
# # print (layer_outs)

In [ ]:
# print(len(layer_outs))

In [ ]:
# from sklearn.manifold import TSNE
# import seaborn as sns
# rt = np.array(layer_outs[13])
# print(rt.shape)
# X_embedded = TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(rt[0])
# print(X_embedded.shape)

In [ ]:
# import matplotlib.pyplot as plt

# fig = plt.gcf()

# # Change seaborn plot size
# fig.set_size_inches(11, 14)
# sns.scatterplot(x = X_embedded[:,0], y = X_embedded[:,1], hue = y_test[:,1], palette = sns.hls_palette(10), legend = 'full')

In [ ]:
# import matplotlib.pyplot as plt

# fig = plt.gcf()

# # Change seaborn plot size
# fig.set_size_inches(11, 14)
# sns.scatterplot(x = X_embedded[:,0], y = X_embedded[:,1], hue = y_test[:,0], palette = sns.hls_palette(20), legend = 'full')